# Opzet eigen Large Language Model met Retrieval Augmented Generation  - LLM met RAG

Vandaag maken we een eigen chatbot mét kennis van lokale documenten. Hieronder staat een stappenplan om deze zelf te bouwen. Maak waar nodig gebruik van een al bestaande chatbot (chatGPT, Mistral, ...) om je code op punt te stellen. Als alles goed is heb je deze daarna (minder hard) nodig...

### Setup

- Creeer een lokale folder met 'local-rag'
- Creëer een virtuele environment, activeer deze
- Installeer chromaDB
- installeer langchain tools, die heb je nodig om je text te splitten
- installeer ollama
- haal het Mistral model binnen - dit kan wel even duren. In de tussentijd kan je best al even verder kijken wat je nog moet doen.
- haal het model binnen om tekst te embedden: nomic-embed-text
- Laat ollama draaien met `ollama serve`

### Webserver
Zet een (Flask of fastapi of andere) webserver op, die volgende routes ondersteunt:
- op de /embed, POST: hier ga je, gebruik makend van een `embed` methode uit de `embed.py` file (zie verder) een document inbedden. Als dit is gelukt geef je een positieve response message terug. Voorzie basic error handling.
- /query, POST: gebruik makend van de `query` methode uit de `query.py`, ga je hier een vraag inlezen.

`jsonify, request, Flask` kunnen nuttige libraries zijn. In essentie is deze webserver gewoon een doorgeefluik voor de volgende codefiles.


### Embedding
**Embedding** in `embed.py` file: voorzie volgende methodes:
- gebruik deze code snippet om enkel pdf's toe te staan:
```python
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in {'pdf'}
```

- Gebruik deze code snippet om een geüploade file op te slagen in een tijdelijke folder:
```python
def save_file(file):
    # Save the uploaded file with a secure filename and return the file path
    ct = datetime.now()
    ts = ct.timestamp()
    filename = str(ts) + "_" + secure_filename(file.filename)
    file_path = os.path.join(TEMP_FOLDER, filename)
    file.save(file_path)

    return file_path
```

- maak een methode `load_and_split_date(file_path)` die een datafile achtereenvolgens
    - inlaadt met `UnstructuredPDFLoader`
    - in stukken splits met een `RecursiveCharacterTextSplitter` - experimenteer met de chunk size en chunk_overlap. Dit bepaalt hoe groot de passages tekst zijn die in de database zullen worden opgeslagen.
    - geef die chunks terug als return van deze methode.

-  een methode `embed(file)`:  Check if the file is valid, save it, load and split the data, add to the database, and remove the temporary file
    - check of de filename geldig is
    - save de file
    - laad deze in en split de data in chunks (met `load_and_split_data`)
    - sla de data op in de vectordatabase (zie verder)
    - delete de tijdelijke data

### Vector database

gebruik deze code voor de vector database. Ze connecteert naadloos met de andere systemen:

```python
import os
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores.chroma import Chroma

CHROMA_PATH = os.getenv('CHROMA_PATH', 'chroma')
COLLECTION_NAME = os.getenv('COLLECTION_NAME', 'local-rag')
TEXT_EMBEDDING_MODEL = os.getenv('TEXT_EMBEDDING_MODEL', 'nomic-embed-text')

def get_vector_db():
    embedding = OllamaEmbeddings(model=TEXT_EMBEDDING_MODEL,show_progress=True)

    db = Chroma(
        collection_name=COLLECTION_NAME,
        persist_directory=CHROMA_PATH,
        embedding_function=embedding
    )

    return db
```

### Query
Tot slot heb je code nodig die op basis van een query extra context gaat zoeken, in de vector database, en deze informatie als context toevoegt aan een query. Implementeer dit op deze manier:
```python
import os
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from get_vector_db import get_vector_db

LLM_MODEL = os.getenv('LLM_MODEL', 'mistral')

def get_prompt():
    QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""Hier kan je infor meegeven over hoe de chatbot zich moet gedragen {question}""",
    )

    template = """Answer the question based ONLY on the following context:
    {context}
    Question: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)

    return QUERY_PROMPT, prompt

# Main function to handle the query process
def query(input):
    if input:
        # Initialize the language model with the specified model name
        llm = ChatOllama(model=LLM_MODEL)
        # Get the vector database instance
        db = get_vector_db()
        # Get the prompt templates
        QUERY_PROMPT, prompt = get_prompt()

        # Set up the retriever to generate multiple queries using the language model and the query prompt
        retriever = MultiQueryRetriever.from_llm(
            db.as_retriever(), 
            llm,
            prompt=QUERY_PROMPT
        )

        # Define the processing chain to retrieve context, generate the answer, and parse the output
        chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

        response = chain.invoke(input)

        return response

    return None

```

Wat nog ontbreekt is de inhoud van de `query(input)` prompt. Ga als volgt tewerk:
- initialiseer een LLM met de geimporteerde chatOllama methode
- gebruik de get_vector_db (hierboven gedefinieerd) om die te kunnen aanroepen
- gebruik de prompt template
- maak een `retriever`, gebruik de `MultiQueryRetriever.from_llm` methode, geef de llm, de database en de prompt mee
- maak een `chain` om alle stukken samen te brengen:
    - context halen met de retriever
    - prompt gebruiken
    - llm aanroepen
    - output parsen met StrOutputParser
- uiteindelijk kan je met `response = chain.invoke(input) ` en `return response` je antwoord teruggeven.